In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_deterministic_transition(grid_length = 25, walls = []):
    matrix = []
    num_states = int(grid_length ** 2)
    for state in range(num_states):
      state_transitions = []
      for action in range(5):
        transitions_given_action = [0] * num_states
        if action == 0: # up
          if state - grid_length < 0 or state - grid_length in walls:
            transitions_given_action[state] = 1
          else:
            transitions_given_action[state - grid_length] = 1

        elif action == 1: # left
          if (state - 1) % grid_length > state % grid_length or \
                              (state - 1)  in walls:
            transitions_given_action[state] = 1
          else:
            transitions_given_action[state - 1] = 1

        elif action == 2: # down
          if state + grid_length > num_states - 1 or state + grid_length in walls:
            transitions_given_action[state] = 1
          else:
            transitions_given_action[state + grid_length] = 1

        elif action == 3: # right
          if (state + 1) % grid_length < state % grid_length or \
                                  (state + 1) in walls:
            transitions_given_action[state] = 1
          else:
            transitions_given_action[state + 1] = 1

        else:
          transitions_given_action[state] = 1
        state_transitions.append(transitions_given_action)
      matrix.append(state_transitions)

    return matrix


In [1]:
def get_transition_matrix(grid_length = 25, walls = [],
                          ):
  num_states = int(grid_length ** 2)


  return get_deterministic_transition(grid_length, walls = walls)


In [ ]:
def get_spectral_gap(grid_length = 25, prob = .2, num_actions = 5,
                     transition_matrix = None,
                    walls = []):

  num_states = int(grid_length) ** 2

  if transition_matrix is None:
    transition_matrix = get_transition_matrix(grid_length, walls = walls)

  pi = np.array([np.array([prob] + [(1-prob)/(num_actions-1)] * (num_actions-1)) for _ in range(num_states)])

  avg_entropy = -np.sum([p * np.log(p) for p in pi[0]])
  transition_matrix = np.array(transition_matrix)
  s, a, _ = transition_matrix.shape

  p = np.zeros((transition_matrix.shape[0], transition_matrix.shape[2]))

  for i in range(transition_matrix.shape[0]):
    for j in range(transition_matrix.shape[2]):
      for k in range(transition_matrix.shape[1]):

        p[i, j] += transition_matrix[i, k, j] * pi[i, k]


  eigenvalues, _ = np.linalg.eig(p)
  sorted_eigenvalues = np.sort(eigenvalues)
  second_largest_eig =  sorted_eigenvalues[-2]

  return avg_entropy, 1 - second_largest_eig

In [ ]:
four_walls_25_25 = [12, 37, 62, 87, 112, 137, 162, 187, 212, 237, 387, 412, 437,
                    462, 487, 512, 537, 562, 587, 612, 300, 301, 302, 303, 304,
                    305, 306, 307, 308, 309, 315, 316, 317, 318, 319, 320, 321,
                    322, 323, 324]


In [ ]:
grid_length = 25



transition_matrix = get_transition_matrix(grid_length,
                                          walls = four_walls_25_25)

deter_data = [get_spectral_gap(grid_length = grid_length, prob = p, transition_matrix = transition_matrix) \
          for p in np.linspace(.20, 1, 100, endpoint=False)]



transition_matrix = get_transition_matrix(grid_length)

deter_data_no_walls = [get_spectral_gap(grid_length = grid_length, prob = p, transition_matrix = transition_matrix) \
          for p in np.linspace(.20, 1, 100, endpoint=False)]

def plot_scatter(data, label):
    x = [point[0] for point in data]
    y = [point[1] for point in data]
    plt.scatter(x, y, label = label)
    plt.xlabel('Policy Entropy')
    plt.ylabel('Spectral Gap of Markov Chain')



plot_scatter(deter_data, '4 walls')

plot_scatter(deter_data_no_walls, 'No walls')
plt.legend()
plt.savefig('eig_ent.png')
plt.show()